In [30]:
import DecisionTreeR
import DecisionTreeLR
import numpy as np
import pandas as pd

In [62]:
class RF():
    def __init__(self,n_estimators=None,estimators = [], max_depth=None,min_sample_split=None):
        self.n_estimators = n_estimators
        self.estimators = estimators
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
    
    #测试集只有单条的预测
    def predict_single(self,treenode,test):
        if treenode.is_leaf:
            return treenode.score
        else:
            #测试集当前节点特征的值小于域值，向训练好的treenode左子树查找
            if test[treenode.feature]<treenode.threshold:
                return self.predict_single(treenode.left,test)
            else:
                return self.predict_single(treenode.right,test)
    
    #测试集多条特征预测
    def predict(self,test):
        result = []
        for i in test:
            single_result = []
            for estimator in self.estimators:
                single_result.append(self.predict_single(estimator,i))
            result.append(np.mean(single_result))
        return result
    
    def fit(self,train,label):
        print(type(self.construct_forest(train,label,depth_left = self.max_depth)))
        self.estimators = self.construct_forest(train,label,depth_left = self.max_depth)
        
    #构建随机森林
    def construct_forest(self,train,label,depth_left):
        tree_nodes = []
        #随机森林终止条件,决策树数量>n_estimator
        for i in range(self.n_estimators):
            tree = DecisionTreeR.Tree(max_depth = depth_left,
                                  min_sample_split = self.min_sample_split,
                                  gamma = 0)
            retrain = self.bootstrap(train)
            retrain = self.refeature(retrain)
            tree.fit(retrain,label)
            print(tree.estimator)
            tree_nodes.append(tree.estimator)
            print(type(tree_nodes))
        return tree_nodes
    
    #随机样本(Bootstrap),有放回得抽取与train样本数量相同的样本
    def bootstrap(self,train):
        index = np.random.choice(range(len(train)),size = len(train) ,replace = True)
        resample = train[index]
        return resample
    
    #随机特征，从n个特征中无放回得随机抽取根号n个特征
    def refeature(self,train):
        #print(int(np.sqrt(len(train[0]))))
        index = np.random.choice(range(len(train[0])),
                size = int(np.sqrt(len(train[0]))) ,replace = False)
        return train[:,index]

In [63]:
def bootstrap(train):
    #print(range(len(train)))
    index = np.random.choice(range(len(train)),size = len(train) ,replace = True)
    resample = train[index]
    return resample

def refeature(train):
    #print(int(np.sqrt(len(train[0]))))
    index = np.random.choice(range(len(train[0])),
            size = int(np.sqrt(len(train[0]))) ,replace = False)
    return train[:,index]

In [64]:
train = np.array(
[
    [1,3,1,3,1],
    [1,1,2,1,3],
    [1,2,2,1,3],
    [1,2,1,1,1],
    [2,1,1,2,2],
    [3,2,2,3,1],
    [3,2,1,2,3],
    [3,1,3,1,1]
])
label =  np.array(
[6,3,2,3,4,2,2,2]
)

In [65]:
tree = DecisionTreeR.Tree(max_depth=100,min_sample_split=1,gamma=0)
tree.fit(train,label)
tree.estimator.is_leaf

False

In [69]:
#bootstrap(train)
#refeature(train)
forest = RF(n_estimators=10,max_depth=10,min_sample_split=1)
forest.fit(train,label)
forest.estimators[0].left.is_leaf

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


False

In [70]:
forest.predict([
    [1,3,1,3,1],
    [1,1,2,1,3]
])

[3.208333333333333, 2.783333333333333]